In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

object_names = ["Plate1", "Plate2", "Plate3", "Plate4", "Plate5"]
folders = ["./datasets/detect/train/images", "./datasets/detect/val/images"]


for folder in folders:
    for object_name in object_names:
        image_files = glob.glob(f"{folder}/image_{object_name}_*.jpg")

        for file in image_files:
            image = cv2.imread(file)
            image[:200,:] = image[0,0]
            image[:,1750:] = image[0,0]

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
        
            kernel = np.ones((5, 5), np.uint8)
            binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            min_area = 5000
            max_area = 200000
            filtered_contours = [cnt for cnt in contours if min_area < cv2.contourArea(cnt) < max_area]

            image_filtered = image.copy()

            yolo_annotations = []
            for contour in filtered_contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image_filtered, (x, y), (x + w, y + h), (0, 255, 0), 2)

                x_center = (x + w / 2) / 1920
                y_center = (y + h / 2) / 1440
                norm_w = w / 1920
                norm_h = h / 1440
                annotation = f"0 {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}"
                yolo_annotations.append(annotation)

            plt.figure(figsize=(8, 6))
            plt.imshow(cv2.cvtColor(image_filtered, cv2.COLOR_BGR2RGB))
            plt.show()

            with open(file.replace("/images\\", "/labels\\").replace(".jpg", ".txt"), "w") as f:
                f.write("\n".join(yolo_annotations))